<a href="https://colab.research.google.com/github/amayaprod/linalchik/blob/main/Amirkhan_Shaybakov_hard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лабораторная работа по линейной алгебре.

Шайбаков Амирхан Анисович, 501983, J3111

#Hard level

Задача (Hard): Принадлежность идеала и его порож-
дающий в кольцах главных идеалов.

Рассмотрим два случая: кольцо целых чисел Z и кольцо по-
линомов от одной переменной K[x], где K — поле (например,
Q или R).

Случай1: Z. На вход подаётся конечный список целых чисел a_1,...,a_k. Они порождают идеал
I = (a1,...,ak) ⊆Z.

Требуется:

• определить, является ли I главным;

• в случае утвердительного ответа, найти порождающий
элемент d∈Z, такой что I = (d).

Случай 2: K[x]. На вход подаётся конечный список полино-
мов f_1(x),...,f_k(x) ∈ K[x], порождающих идеал
I = (f_1,...,f_k) ⊆ K[x].

Требуется:

• определить, является ли I главным;

• вслучаеутвердительногоответа,найтипорождающийпо-
лином d(x) ∈ K[x], такой что I = (d(x)).

Подсказка. В голове крутится одно слово, Евкл...

Отгадка к подсказке: Алгоритм Евклида.

Как работает Алгоритм Евклида:

1)Берём два числа: 𝑎 и 𝑏, где 𝑎>𝑏.

2)Делим 𝑎 на 𝑏 и берём остаток 𝑟.

3)Заменяем пару (𝑎,𝑏) на (𝑏,𝑟).

4)Повторяем деление, пока остаток не станет равен нулю.

5)Последнее ненулевое 𝑏 — это и есть НОД.

In [6]:
from typing import Union, List
import math
from functools import reduce


class RingElement:
    """Базовый класс для элементов колец: целых чисел и полиномов."""

    def __init__(self, data: Union[int, List[float]]):
        """
        data:
        - если это целое число -> элемент кольца Z;
        - если это список коэффициентов [a0, a1, ..., an] -> полином a0 + a1*x + ... + an*x^n.
        """
        self.data = data
        self.is_polynomial = isinstance(data, list)

    def __repr__(self) -> str:
        if self.is_polynomial:
            terms = [f"{c}*x^{i}" if i > 0 else str(c)
                     for i, c in enumerate(self.data) if c != 0]
            return " + ".join(terms) if terms else "0"
        return str(self.data)

    def is_multiple_of(self, other: 'RingElement') -> bool:
        if self.is_polynomial != other.is_polynomial:
            raise ValueError("Нельзя сравнивать элементы разных типов")

        if not self.is_polynomial:
            return self.data % other.data == 0
        else:
            # Для полиномов проверяем, что остаток от деления нулевой
            remainder = self.poly_mod(other)
            return len(remainder) == 0 or all(abs(coef) < 1e-10 for coef in remainder)

    def poly_mod(self, other: 'RingElement') -> List[float]:
        """Деление полиномов с остатком"""
        if not self.is_polynomial or not other.is_polynomial:
            raise ValueError("Метод только для полиномов")

        f = self.data.copy()
        g = other.data.copy()

        # Удаляем ведущие нули
        while len(f) > 0 and abs(f[-1]) < 1e-10:
            f.pop()
        while len(g) > 0 and abs(g[-1]) < 1e-10:
            g.pop()

        if len(g) == 0:
            raise ValueError("Деление на нулевой полином")

        # Алгоритм деления полиномов
        len_f, len_g = len(f), len(g)
        if len_f < len_g:
            return f  # остаток равен f

        result = f.copy()
        for i in range(len_f - len_g, -1, -1):
            if abs(result[i + len_g - 1]) < 1e-10:
                continue
            factor = result[i + len_g - 1] / g[-1]
            for j in range(len_g):
                result[i + j] -= factor * g[j]

        # Обрезаем нули в конце
        while len(result) > 0 and abs(result[-1]) < 1e-10:
            result.pop()

        return result


def gcd_two_integers(a: int, b: int) -> int:
    """Алгоритм Евклида для двух целых чисел"""
    a, b = abs(a), abs(b)
    while b != 0:
        a, b = b, a % b
    return a


def polynomial_division(f: List[float], g: List[float]) -> tuple[List[float], List[float]]:
    """Корректное деление полиномов с возвратом частного и остатка"""
    if all(abs(coef) < 1e-10 for coef in g):
        raise ValueError("Деление на нулевой полином")

    # Создаем копии
    dividend = f.copy()
    divisor = g.copy()

    # Удаляем только ведущие нули (сохраняем структуру полинома)
    while len(dividend) > 0 and abs(dividend[-1]) < 1e-10:
        dividend.pop()
    while len(divisor) > 0 and abs(divisor[-1]) < 1e-10:
        divisor.pop()

    if len(dividend) < len(divisor):
        return [0.0], dividend  # частное = 0, остаток = dividend

    len_diff = len(dividend) - len(divisor)
    quotient = [0.0] * (len_diff + 1)
    remainder = dividend.copy()

    for i in range(len_diff, -1, -1):
        if len(remainder) < i + len(divisor):
            continue

        if abs(remainder[i + len(divisor) - 1]) < 1e-10:
            continue

        factor = remainder[i + len(divisor) - 1] / divisor[-1]
        quotient[i] = factor

        for j in range(len(divisor)):
            remainder[i + j] -= factor * divisor[j]

    # Удаляем ведущие нули из остатка
    while len(remainder) > 0 and abs(remainder[-1]) < 1e-10:
        remainder.pop()

    return quotient, remainder


def gcd_two_polynomials(f: List[float], g: List[float]) -> List[float]:
    """Алгоритм Евклида для двух полиномов, соответствующий требованиям"""
    if all(abs(coef) < 1e-10 for coef in g):
        return f

    a, b = f.copy(), g.copy()

    # Удаляем ведущие нули
    while len(a) > 0 and abs(a[-1]) < 1e-10:
        a.pop()
    while len(b) > 0 and abs(b[-1]) < 1e-10:
        b.pop()

    while len(b) > 0 and not all(abs(coef) < 1e-10 for coef in b):
        _, remainder = polynomial_division(a, b)
        a, b = b, remainder

    # Нормализация - делаем старший коэффициент 1
    if len(a) > 0 and abs(a[-1]) > 1e-10:
        a = [coef / a[-1] for coef in a]

    return a

def poly_division_remainder(f: List[float], g: List[float]) -> List[float]:
    """Корректное деление полиномов с остатком"""
    if len(g) == 0 or all(abs(coef) < 1e-10 for coef in g):
        raise ValueError("Деление на нулевой полином")

    f = f.copy()
    g = g.copy()

    # Удаляем ведущие нули
    while len(f) > 0 and abs(f[-1]) < 1e-10:
        f.pop()
    while len(g) > 0 and abs(g[-1]) < 1e-10:
        g.pop()

    if len(f) < len(g):
        return f

    remainder = f.copy()
    len_f, len_g = len(f), len(g)

    for i in range(len_f - len_g, -1, -1):
        if len(remainder) < i + len_g:
            continue

        if abs(remainder[i + len_g - 1]) < 1e-10:
            continue

        factor = remainder[i + len_g - 1] / g[-1]

        for j in range(len_g):
            if i + j < len(remainder):
                remainder[i + j] -= factor * g[j]

    # Обрезаем нули в конце
    while len(remainder) > 0 and abs(remainder[-1]) < 1e-10:
        remainder.pop()

    return remainder


def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
    """
    Возвращает порождающий главного идеала, порождённого заданными элементами.
    - Для Z: НОД целых чисел.
    - Для K[x]: НОД полиномов (с коэффициентами в поле K).
    """
    if not elements:
        raise ValueError("Список элементов не может быть пустым")

    # Проверяем, что все элементы одного типа
    first_type = elements[0].is_polynomial
    for elem in elements[1:]:
        if elem.is_polynomial != first_type:
            raise ValueError("Все элементы должны быть одного типа")

    if not first_type:
        # Случай целых чисел
        numbers = [elem.data for elem in elements]
        if len(numbers) == 1:
            gcd_val = abs(numbers[0])
        else:
            gcd_val = reduce(gcd_two_integers, numbers)
        return RingElement(gcd_val)
    else:
        # Случай полиномов
        polynomials = [elem.data for elem in elements]
        if len(polynomials) == 1:
            gcd_poly = polynomials[0]
        else:
            gcd_poly = reduce(gcd_two_polynomials, polynomials)

        # Нормализуем полином
        if len(gcd_poly) > 0 and abs(gcd_poly[-1]) > 1e-10:
            gcd_poly = [coef / gcd_poly[-1] for coef in gcd_poly]

        return RingElement(gcd_poly)


def is_element_in_ideal(element: RingElement, ideal_generators: List[RingElement]) -> bool:
    """
    Проверяет, принадлежит ли элемент идеалу, порожденному заданными образующими.

    Используется свойство: если I = (d) - главный идеал, то f ∈ I тогда и только тогда,
    когда d делит f (для целых чисел) или d(x) делит f(x) (для полиномов).
    """
    d = gcd_ring_elements(ideal_generators)
    return element.is_multiple_of(d)


# ДЕМОНСТРАЦИЯ РАБОТЫ
if __name__ == "__main__":
    print("ДЕМОНСТРАЦИЯ РАБОТЫ КЛАССА RingElement")
    print("=" * 50)

    # Пример 1: Целые числа
    print("Пример 1: Целые числа")
    numbers = [RingElement(12), RingElement(18), RingElement(24)]
    print(f"Идеал I = ({', '.join(map(str, numbers))})")

    d = gcd_ring_elements(numbers)
    print(f"Порождающий идеала: {d}")
    print(f"Идеал I = ({d})")

    # Проверка принадлежности элементов идеалу
    test_elements = [RingElement(6), RingElement(30), RingElement(7)]
    for elem in test_elements:
        belongs = is_element_in_ideal(elem, numbers)
        print(f"Элемент {elem} ∈ I: {belongs}")

    print("\n" + "=" * 50)

    # Пример 2: Полиномы
    print("Пример 2: Полиномы")
    # f1 = x² - 1, f2 = x² - 2x + 1
    polynomials = [
        RingElement([-1, 0, 1]),  # x² - 1
        RingElement([1, -2, 1])  # x² - 2x + 1
    ]
    print(f"Идеал I = ({', '.join(map(str, polynomials))})")

    d_poly = gcd_ring_elements(polynomials)
    print(f"Порождающий идеала: {d_poly}")

    # Проверка принадлежности полиномов идеалу
    test_polys = [
        RingElement([-1, 1]),  # x - 1
        RingElement([1, 1]),  # x + 1
        RingElement([2, -2, 0])  # 2x² - 2x
    ]
    for poly in test_polys:
        belongs = is_element_in_ideal(poly, polynomials)
        print(f"Полином {poly} ∈ I: {belongs}")

    print("\n" + "=" * 50)

    # Пример 3: Более сложный случай с полиномами
    print("Пример 3: Сложные полиномы")
    complex_polys = [
        RingElement([0, -1, 0, 1]),  # x³ - x
        RingElement([-1, 0, 1])  # x² - 1
    ]
    print(f"Идеал I = ({', '.join(map(str, complex_polys))})")

    d_complex = gcd_ring_elements(complex_polys)
    print(f"Порождающий идеала: {d_complex}")

    # Проверка
    test_complex = RingElement([0, -1, 1])  # x² - x
    belongs = is_element_in_ideal(test_complex, complex_polys)
    print(f"Полином {test_complex} ∈ I: {belongs}")

ДЕМОНСТРАЦИЯ РАБОТЫ КЛАССА RingElement
Пример 1: Целые числа
Идеал I = (12, 18, 24)
Порождающий идеала: 6
Идеал I = (6)
Элемент 6 ∈ I: True
Элемент 30 ∈ I: True
Элемент 7 ∈ I: False

Пример 2: Полиномы
Идеал I = (-1 + 1*x^2, 1 + -2*x^1 + 1*x^2)
Порождающий идеала: -1.0 + 1.0*x^1
Полином -1 + 1*x^1 ∈ I: True
Полином 1 + 1*x^1 ∈ I: False
Полином 2 + -2*x^1 ∈ I: True

Пример 3: Сложные полиномы
Идеал I = (-1*x^1 + 1*x^3, -1 + 1*x^2)
Порождающий идеала: -1.0 + 1.0*x^2
Полином -1*x^1 + 1*x^2 ∈ I: False


Как знание порождающего помогает определить принадлежность элемента идеалу?

Если мы знаем, что идеал I = (d) главный и нашли его порождающий d, то для любого элемента f:

f ∈ I тогда и только тогда, когда f делится на d без остатка
Это следует из определения главного идеала: все элементы идеала — это кратные элемента d. Поэтому проверка принадлежности сводится к проверке делимости на d.